In [49]:
import numpy as np
import json
import pandas as pd
import requests
import time
import pickle
import matplotlib.pyplot as plt

from urllib import parse
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

In [4]:
with open('datas_practice_5.pickle', 'rb') as f:
    datas = pickle.load(f)

In [45]:
api_key = "RGAPI-69bc0d5a-5d4d-49e7-9175-8c12232b8abb" # 주기적으로 갱신

In [11]:
request_header = {
                    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.164 Safari/537.36",
                    "Accept-Language": "ko,en-US;q=0.9,en;q=0.8,es;q=0.7",
                    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
                    "Origin": "https://developer.riotgames.com",
                    "X-Riot-Token": api_key
                }

In [12]:
request_header_match_v5 = {
                    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36 Edg/115.0.1901.203",
                    "Accept-Language": "ko,en;q=0.9,en-US;q=0.8",
                    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
                    "Origin": "https://developer.riotgames.com"
}

In [13]:
def Champion_mastery_v4(encryptedSummonerId):
    url = f"https://kr.api.riotgames.com/lol/champion-mastery/v4/champion-masteries/by-summoner/{encryptedSummonerId}?api_key=api_key"
    
    response = requests.get(url, headers=request_header)
    
    # Rate Limit 초과 오류를 처리
    if response.status_code == 429:
        retry_after = int(response.headers['Retry-After'])
        print(f"Rate Limit 초과, {retry_after}초 대기 후 재시도")
        time.sleep(retry_after)
        response = requests.get(url, headers=request_header)
    
    return response.json()

In [14]:
def summoner_v4_by_summoner_name(summonerName):
    # encodingSummonerName = parse.quote(summonerName)
    url = f"https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/{summonerName}"
    
    # Rate Limit을 준수하기 위해 2초 대기
    time.sleep(2)
    
    response = requests.get(url, headers=request_header)
    
    # Rate Limit 초과 오류를 처리
    if response.status_code == 429:
        retry_after = int(response.headers['Retry-After'])
        print(f"Rate Limit 초과, {retry_after}초 대기 후 재시도")
        time.sleep(retry_after)
        response = requests.get(url, headers=request_header)
    
    return response.json()

In [15]:
def summoner_v4_by_encryptedSummonerId(summonerId):
    url = f"https://kr.api.riotgames.com/lol/summoner/v4/summoners/{summonerId}?api_key={api_key}"
    
    time.sleep(2)
    
    response = requests.get(url, headers=request_header)
    
    # Rate Limit 초과 오류를 처리
    if response.status_code == 429:
        retry_after = int(response.headers['Retry-After'])
        print(f"Rate Limit 초과, {retry_after}초 대기 후 재시도")
        time.sleep(retry_after)
        response = requests.get(url, headers=request_header)
        
    return response.json()

In [16]:
def summoner_v4_by_encryptedSummonerId_champion(summonerId):
    url = f"https://kr.api.riotgames.com/lol/champion-mastery/v4/champion-masteries/by-summoner/{summonerId}?api_key={api_key}"
    
    time.sleep(2)
    
    response = requests.get(url, headers=request_header_match_v5)
    
    # Rate Limit 초과 오류를 처리
    if response.status_code == 429:
        retry_after = int(response.headers['Retry-After'])
        print(f"Rate Limit 초과, {retry_after}초 대기 후 재시도")
        time.sleep(retry_after)
        response = requests.get(url, headers=request_header_match_v5)
        
    return response.json()

In [17]:
def match_v5_puuid(puuid):
    url =f"https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count=5&api_key={api_key}"
    
    time.sleep(2)
    
    response = requests.get(url, headers = request_header_match_v5)
    
    # Rate Limit 초과 오류를 처리
    if response.status_code == 429:
        retry_after = int(response.headers['Retry-After'])
        print(f"Rate Limit 초과, {retry_after}초 대기 후 재시도")
        time.sleep(retry_after)
        response = requests.get(url, headers=request_header_match_v5)
        
    return response.json()

In [18]:
def match_v5_matchid(matchid):
    url = f"https://asia.api.riotgames.com/lol/match/v5/matches/{matchid}?api_key={api_key}"
    
    time.sleep(2)
    
    response = requests.get(url, headers = request_header_match_v5)
    
    # Rate Limit 초과 오류를 처리
    if response.status_code == 429:
        retry_after = int(response.headers['Retry-After'])
        print(f"Rate Limit 초과, {retry_after}초 대기 후 재시도")
        time.sleep(retry_after)
        response = requests.get(url, headers=request_header_match_v5)
        
    return response.json()

In [19]:
from datetime import datetime
import pytz

def convert_milliseconds_to_new_york_time(milliseconds):
    # 밀리초를 초로 변환
    seconds = milliseconds / 1000

    # 뉴욕 시간대 객체 생성
    new_york_tz = pytz.timezone('America/New_York')

    # 밀리초로부터 날짜 계산
    input_date = datetime.fromtimestamp(seconds, tz=new_york_tz)

    return input_date

def get_today_date_in_new_york():
    # 뉴욕 시간대 객체 생성
    new_york_tz = pytz.timezone('America/New_York')

    # 현재 날짜를 뉴욕 시간대로 얻기
    today_date = datetime.now(tz=new_york_tz)

    return today_date

In [20]:
from datetime import datetime
import pytz

def convert_milliseconds_to_new_york_time(milliseconds):
    # 밀리초를 초로 변환
    seconds = milliseconds / 1000

    # 뉴욕 시간대 객체 생성
    new_york_tz = pytz.timezone('America/New_York')

    # 밀리초로부터 날짜 계산
    input_date = datetime.fromtimestamp(seconds, tz=new_york_tz)

    return input_date

def get_today_date_in_new_york():
    # 뉴욕 시간대 객체 생성
    new_york_tz = pytz.timezone('America/New_York')

    # 현재 날짜를 뉴욕 시간대로 얻기
    today_date = datetime.now(tz=new_york_tz)

    return today_date

In [ ]:
for i in range(10, len(datas)):
    print(i)
    s1 = summoner_v4_by_summoner_name(datas[i]['user_name'])
    datas[i]['summoner_level'] = s1['summonerLevel']

10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105


In [21]:
import requests
import time

def get_all_league_exp_v4_data(api_key, region='kr'):
    
    base_url = f'https://{region}.api.riotgames.com/lol/league-exp/v4/entries/'

    queues = ['RANKED_SOLO_5x5']
    # ['IRON', 'BRONZE', 'SILVER', 'GOLD', 'PLATINUM', 'DIAMOND']
    tiers = ['IRON', 'BRONZE', 'SILVER', 'GOLD', 'PLATINUM', 'DIAMOND', 'MASTER', 'GRANDMASTER', 'CHALLENGER']
    
    divisions = ['I', 'II', 'III', 'IV']

    all_data = []

    for queue in queues:
        for tier in tiers:
            for division in divisions:
                page = 26
                while page <= 70:  # 각 디비전 페이지를 10페이지까지만 추출
                    print(f'Fetching {queue}/{tier}/{division} Page {page}...')
                    params = {
                        'api_key': api_key,
                        'page': page,
                        'pageSize': 200,  # 페이지당 항목 수
                    }
                    response = requests.get(f'{base_url}{queue}/{tier}/{division}', params=params)

                    if response.status_code == 429:
                        # Rate Limit 초과 시 대기
                        retry_after = int(response.headers['Retry-After'])
                        time.sleep(retry_after)
                        continue

                    data = response.json()
                    all_data.extend(data)

                    if len(data) < 200:
                        break
                    else:
                        page += 1

    return all_data

In [22]:
all_data = get_all_league_exp_v4_data(api_key)

Fetching RANKED_SOLO_5x5/IRON/I Page 26...
Fetching RANKED_SOLO_5x5/IRON/I Page 27...
Fetching RANKED_SOLO_5x5/IRON/I Page 28...
Fetching RANKED_SOLO_5x5/IRON/I Page 29...
Fetching RANKED_SOLO_5x5/IRON/I Page 30...
Fetching RANKED_SOLO_5x5/IRON/I Page 31...
Fetching RANKED_SOLO_5x5/IRON/I Page 32...
Fetching RANKED_SOLO_5x5/IRON/I Page 33...
Fetching RANKED_SOLO_5x5/IRON/I Page 34...
Fetching RANKED_SOLO_5x5/IRON/I Page 35...
Fetching RANKED_SOLO_5x5/IRON/I Page 36...
Fetching RANKED_SOLO_5x5/IRON/I Page 37...
Fetching RANKED_SOLO_5x5/IRON/I Page 38...
Fetching RANKED_SOLO_5x5/IRON/I Page 39...
Fetching RANKED_SOLO_5x5/IRON/I Page 40...
Fetching RANKED_SOLO_5x5/IRON/I Page 41...
Fetching RANKED_SOLO_5x5/IRON/I Page 42...
Fetching RANKED_SOLO_5x5/IRON/I Page 43...
Fetching RANKED_SOLO_5x5/IRON/I Page 44...
Fetching RANKED_SOLO_5x5/IRON/I Page 45...
Fetching RANKED_SOLO_5x5/IRON/I Page 46...
Fetching RANKED_SOLO_5x5/IRON/I Page 47...
Fetching RANKED_SOLO_5x5/IRON/I Page 48...
Fetching RA

Fetching RANKED_SOLO_5x5/BRONZE/I Page 41...
Fetching RANKED_SOLO_5x5/BRONZE/I Page 42...
Fetching RANKED_SOLO_5x5/BRONZE/I Page 43...
Fetching RANKED_SOLO_5x5/BRONZE/I Page 44...
Fetching RANKED_SOLO_5x5/BRONZE/I Page 45...
Fetching RANKED_SOLO_5x5/BRONZE/I Page 46...
Fetching RANKED_SOLO_5x5/BRONZE/I Page 47...
Fetching RANKED_SOLO_5x5/BRONZE/I Page 48...
Fetching RANKED_SOLO_5x5/BRONZE/I Page 49...
Fetching RANKED_SOLO_5x5/BRONZE/I Page 50...
Fetching RANKED_SOLO_5x5/BRONZE/I Page 51...
Fetching RANKED_SOLO_5x5/BRONZE/I Page 52...
Fetching RANKED_SOLO_5x5/BRONZE/I Page 53...
Fetching RANKED_SOLO_5x5/BRONZE/I Page 54...
Fetching RANKED_SOLO_5x5/BRONZE/I Page 55...
Fetching RANKED_SOLO_5x5/BRONZE/I Page 56...
Fetching RANKED_SOLO_5x5/BRONZE/I Page 56...
Fetching RANKED_SOLO_5x5/BRONZE/I Page 57...
Fetching RANKED_SOLO_5x5/BRONZE/I Page 58...
Fetching RANKED_SOLO_5x5/BRONZE/I Page 59...
Fetching RANKED_SOLO_5x5/BRONZE/I Page 60...
Fetching RANKED_SOLO_5x5/BRONZE/I Page 61...
Fetching R

Fetching RANKED_SOLO_5x5/SILVER/I Page 38...
Fetching RANKED_SOLO_5x5/SILVER/I Page 39...
Fetching RANKED_SOLO_5x5/SILVER/I Page 40...
Fetching RANKED_SOLO_5x5/SILVER/I Page 41...
Fetching RANKED_SOLO_5x5/SILVER/I Page 42...
Fetching RANKED_SOLO_5x5/SILVER/I Page 43...
Fetching RANKED_SOLO_5x5/SILVER/I Page 44...
Fetching RANKED_SOLO_5x5/SILVER/I Page 45...
Fetching RANKED_SOLO_5x5/SILVER/I Page 46...
Fetching RANKED_SOLO_5x5/SILVER/I Page 47...
Fetching RANKED_SOLO_5x5/SILVER/I Page 48...
Fetching RANKED_SOLO_5x5/SILVER/I Page 49...
Fetching RANKED_SOLO_5x5/SILVER/I Page 50...
Fetching RANKED_SOLO_5x5/SILVER/I Page 51...
Fetching RANKED_SOLO_5x5/SILVER/I Page 52...
Fetching RANKED_SOLO_5x5/SILVER/I Page 53...
Fetching RANKED_SOLO_5x5/SILVER/I Page 54...
Fetching RANKED_SOLO_5x5/SILVER/I Page 55...
Fetching RANKED_SOLO_5x5/SILVER/I Page 56...
Fetching RANKED_SOLO_5x5/SILVER/I Page 57...
Fetching RANKED_SOLO_5x5/SILVER/I Page 58...
Fetching RANKED_SOLO_5x5/SILVER/I Page 59...
Fetching R

Fetching RANKED_SOLO_5x5/GOLD/I Page 35...
Fetching RANKED_SOLO_5x5/GOLD/I Page 36...
Fetching RANKED_SOLO_5x5/GOLD/I Page 37...
Fetching RANKED_SOLO_5x5/GOLD/I Page 38...
Fetching RANKED_SOLO_5x5/GOLD/I Page 39...
Fetching RANKED_SOLO_5x5/GOLD/I Page 40...
Fetching RANKED_SOLO_5x5/GOLD/I Page 41...
Fetching RANKED_SOLO_5x5/GOLD/I Page 42...
Fetching RANKED_SOLO_5x5/GOLD/I Page 43...
Fetching RANKED_SOLO_5x5/GOLD/I Page 44...
Fetching RANKED_SOLO_5x5/GOLD/I Page 45...
Fetching RANKED_SOLO_5x5/GOLD/I Page 46...
Fetching RANKED_SOLO_5x5/GOLD/I Page 47...
Fetching RANKED_SOLO_5x5/GOLD/I Page 48...
Fetching RANKED_SOLO_5x5/GOLD/I Page 49...
Fetching RANKED_SOLO_5x5/GOLD/I Page 50...
Fetching RANKED_SOLO_5x5/GOLD/I Page 51...
Fetching RANKED_SOLO_5x5/GOLD/I Page 52...
Fetching RANKED_SOLO_5x5/GOLD/I Page 53...
Fetching RANKED_SOLO_5x5/GOLD/I Page 54...
Fetching RANKED_SOLO_5x5/GOLD/I Page 55...
Fetching RANKED_SOLO_5x5/GOLD/I Page 56...
Fetching RANKED_SOLO_5x5/GOLD/I Page 57...
Fetching RA

Fetching RANKED_SOLO_5x5/PLATINUM/I Page 59...
Fetching RANKED_SOLO_5x5/PLATINUM/I Page 60...
Fetching RANKED_SOLO_5x5/PLATINUM/I Page 61...
Fetching RANKED_SOLO_5x5/PLATINUM/I Page 62...
Fetching RANKED_SOLO_5x5/PLATINUM/I Page 63...
Fetching RANKED_SOLO_5x5/PLATINUM/I Page 64...
Fetching RANKED_SOLO_5x5/PLATINUM/I Page 65...
Fetching RANKED_SOLO_5x5/PLATINUM/I Page 66...
Fetching RANKED_SOLO_5x5/PLATINUM/I Page 67...
Fetching RANKED_SOLO_5x5/PLATINUM/I Page 68...
Fetching RANKED_SOLO_5x5/PLATINUM/I Page 69...
Fetching RANKED_SOLO_5x5/PLATINUM/I Page 70...
Fetching RANKED_SOLO_5x5/PLATINUM/II Page 26...
Fetching RANKED_SOLO_5x5/PLATINUM/II Page 27...
Fetching RANKED_SOLO_5x5/PLATINUM/II Page 28...
Fetching RANKED_SOLO_5x5/PLATINUM/II Page 29...
Fetching RANKED_SOLO_5x5/PLATINUM/II Page 30...
Fetching RANKED_SOLO_5x5/PLATINUM/II Page 31...
Fetching RANKED_SOLO_5x5/PLATINUM/II Page 32...
Fetching RANKED_SOLO_5x5/PLATINUM/II Page 33...
Fetching RANKED_SOLO_5x5/PLATINUM/II Page 34...
Fetc

Fetching RANKED_SOLO_5x5/DIAMOND/II Page 28...
Fetching RANKED_SOLO_5x5/DIAMOND/II Page 29...
Fetching RANKED_SOLO_5x5/DIAMOND/II Page 30...
Fetching RANKED_SOLO_5x5/DIAMOND/II Page 31...
Fetching RANKED_SOLO_5x5/DIAMOND/II Page 32...
Fetching RANKED_SOLO_5x5/DIAMOND/II Page 33...
Fetching RANKED_SOLO_5x5/DIAMOND/II Page 34...
Fetching RANKED_SOLO_5x5/DIAMOND/II Page 35...
Fetching RANKED_SOLO_5x5/DIAMOND/II Page 36...
Fetching RANKED_SOLO_5x5/DIAMOND/II Page 37...
Fetching RANKED_SOLO_5x5/DIAMOND/II Page 37...
Fetching RANKED_SOLO_5x5/DIAMOND/II Page 38...
Fetching RANKED_SOLO_5x5/DIAMOND/II Page 39...
Fetching RANKED_SOLO_5x5/DIAMOND/II Page 40...
Fetching RANKED_SOLO_5x5/DIAMOND/II Page 41...
Fetching RANKED_SOLO_5x5/DIAMOND/II Page 42...
Fetching RANKED_SOLO_5x5/DIAMOND/II Page 43...
Fetching RANKED_SOLO_5x5/DIAMOND/II Page 44...
Fetching RANKED_SOLO_5x5/DIAMOND/II Page 45...
Fetching RANKED_SOLO_5x5/DIAMOND/II Page 46...
Fetching RANKED_SOLO_5x5/DIAMOND/II Page 47...
Fetching RANK

In [23]:
len(all_data)

210749

In [24]:
datas = []

for idx, st in enumerate(all_data):
    u_data = {}
    try:
        u_data['user_name'] = st['summonerName']
        u_data['play_num'] = st['wins'] + st['losses']
        u_data['win_rate'] = 100 * (st['wins'] / (st['wins'] + st['losses']))
        u_data['veteran'] = st['veteran']
        u_data['hot_streak'] = st['hotStreak']
        u_data['freshblood'] = st['freshBlood']
        u_data['league_points'] = st['leaguePoints']
        u_data['summonerId'] = st['summonerId']
        u_data['inactive'] = st['inactive']
        u_data['tier'] = st['tier']
        u_data['summonerId'] = st['summonerId']
        datas.append(u_data)
        
    except Exception as e:
        print(st)
        continue

status
status
status
status
status
status
status
status
status
status
status


In [25]:
len(datas)

210738

In [64]:
datas

[{'user_name': '인생이란이런거지',
  'play_num': 16,
  'win_rate': 37.5,
  'veteran': False,
  'hot_streak': False,
  'freshblood': True,
  'league_points': 0,
  'summonerId': '97jdnpEQYg0QEwbOQEb3w6usZKaKvl1cWh5fpYPhq8JSis5v',
  'inactive': False,
  'tier': 'IRON',
  'revisionDate': 1694854484010,
  'summonerLevel': 56,
  'puuid': '7x1gvTaX2XdqR2ZWcj_ZMX2eGIH_Ys4FU4WxHyqIPHV2hznVrGIrg4-sqXa7O9MAYCA9lbaGl1hhDw'},
 {'user_name': '냄비에데임',
  'play_num': 5,
  'win_rate': 40.0,
  'veteran': False,
  'hot_streak': False,
  'freshblood': False,
  'league_points': 15,
  'summonerId': 'rZaNOe7FTkeeVGVgXWow5b1KpElHuB0x5JLeq22fhqp6XDdE',
  'inactive': False,
  'tier': 'IRON',
  'revisionDate': 1694866558000,
  'summonerLevel': 291,
  'puuid': 'tmwn_OJnbuRKJ-AbYIPis2a90cpkg8QmOsEuTVibVVUpw609ype3p0CaxP0lnJrDK1c6pkRn9Sg9Iw'},
 {'user_name': 'I SKT TI Faker I',
  'play_num': 10,
  'win_rate': 30.0,
  'veteran': False,
  'hot_streak': False,
  'freshblood': False,
  'league_points': 17,
  'summonerId': 'ASRg

In [47]:
count = 0
for i in range(0, len(datas)):
    if datas[i]['inactive'] == True:
        count += 1
    else:
        continue

In [48]:
count

0

## get_summoner_info(summoner_name)

In [21]:
def get_summoner_info(api_key, summoner_name, region='kr'):
    base_url = f'https://{region}.api.riotgames.com/lol/summoner/v4/summoners/by-name/{summoner_name}'
    params = {'api_key': api_key}

    while True:
        try:
            response = requests.get(base_url, params=params)

            if response.status_code == 200:
                summoner_info = response.json()
                return summoner_info
            elif response.status_code == 429:
                # Rate Limit 초과 오류인 경우 대기 후 재시도
                retry_after = int(response.headers.get("Retry-After", 5))
                print(f"Rate Limit 초과, {retry_after} 초 대기 후 다시 시도합니다.")
                time.sleep(retry_after)
                response = requests.get(base_url, headers=headers)
            else:
                print(f"오류 발생: {response.status_code}")
                return None

        except Exception as e:
            print(f"요청 중 오류 발생: {str(e)}")
            return None

## get_summoner_info(summonerId)

In [30]:
def get_summoner_info_by_id(api_key, summoner_id, region='kr'):
    base_url = f'https://{region}.api.riotgames.com/lol/summoner/v4/summoners/{summoner_id}'
    params = {'api_key': api_key}

    try:
        response = requests.get(base_url, params=params)

        if response.status_code == 200:
            summoner_info = response.json()
            return summoner_info
        else:
            print(f"오류 발생: {response.status_code}")
            return None

    except Exception as e:
        print(f"요청 중 오류 발생: {str(e)}")
        return None

In [23]:
for i in range(167202, len(datas)):
    print(i)
    try:
        summoner_info = get_summoner_info(api_key, datas[i]['user_name'])
        datas[i]['revisionDate'] = summoner_info['revisionDate']
        datas[i]['summonerLevel'] = summoner_info['summonerLevel']
        datas[i]['puuid'] = summoner_info['puuid']
        
    except TypeError as e:
        print(f'오류 발생 위치 : {i}')
        print('변경 전 닉네임 :', {datas[i]['user_name']})
        datas[i]['user_name'] = summoner_v4_by_encryptedSummonerId(datas[i]['summonerId'])['name']
        print('변경 후 닉네임 :', {datas[i]['user_name']})
        
        try:
            summoner_info = get_summoner_info(api_key, datas[i]['user_name'])
            datas[i]['revisionDate'] = summoner_info['revisionDate']
            datas[i]['summonerLevel'] = summoner_info['summonerLevel']
            datas[i]['puuid'] = summoner_info['puuid']
            
        except TypeError as e:
            print(f"닉네임 조회 실패로 summonerId로 조회 실행: {datas[i]['summonerId']}")
            summoner_info = get_summoner_info_by_id(api_key, datas[i]['summonerId'])
            datas[i]['revisionDate'] = summoner_info['revisionDate']
            datas[i]['summonerLevel'] = summoner_info['summonerLevel']
            datas[i]['puuid'] = summoner_info['puuid']

167202
오류 발생: 404
오류 발생 위치 : 167202
변경 전 닉네임 : {'ZZ7'}
변경 후 닉네임 : {'YOON22'}
167203
167204
167205
167206
167207
167208
167209
167210
167211
167212
167213
167214
167215
167216
167217
167218
167219
167220
167221
167222
167223
167224
167225
167226
167227
167228
167229
167230
167231
167232
167233
167234
167235
167236
167237
167238
167239
167240
167241
167242
167243
167244
167245
167246
167247
167248
167249
167250
167251
167252
167253
167254
167255
167256
167257
167258
167259
167260
167261
167262
167263
오류 발생: 404
오류 발생 위치 : 167263
변경 전 닉네임 : {'회계학과 봉재민'}
변경 후 닉네임 : {'학회장 임형범'}
167264
167265
167266
167267
167268
167269
167270
167271
167272
167273
167274
167275
167276
167277
오류 발생: 404
오류 발생 위치 : 167277
변경 전 닉네임 : {'유틸폿대가리깸'}
변경 후 닉네임 : {'침묵고수'}
167278
167279
167280
167281
167282
167283
167284
167285
167286
167287
167288
오류 발생: 404
오류 발생 위치 : 167288
변경 전 닉네임 : {'하얀천이랑 바람'}
변경 후 닉네임 : {'13342080del'}
167289
167290
167291
167292
167293
167294
Rate Limit 초과, 88 초 대기 후 다시 시도합니다.
요청 중 오류 발생: name

168015
오류 발생: 404
오류 발생 위치 : 168015
변경 전 닉네임 : {'개상윤9999'}
변경 후 닉네임 : {'개상윤'}
168016
168017
168018
168019
168020
168021
168022
168023
168024
168025
168026
168027
168028
168029
168030
168031
168032
168033
168034
168035
168036
168037
168038
168039
168040
168041
168042
168043
168044
168045
168046
168047
168048
168049
168050
168051
168052
168053
168054
Rate Limit 초과, 90 초 대기 후 다시 시도합니다.
요청 중 오류 발생: name 'headers' is not defined
오류 발생 위치 : 168054
변경 전 닉네임 : {'상하01 도서관'}
변경 후 닉네임 : {'상하01 도서관'}
168055
168056
168057
168058
168059
168060
168061
168062
168063
168064
168065
168066
168067
168068
168069
168070
168071
168072
168073
168074
168075
168076
168077
168078
168079
168080
168081
168082
168083
168084
168085
168086
168087
168088
168089
168090
168091
168092
168093
168094
168095
168096
168097
168098
168099
오류 발생: 404
오류 발생 위치 : 168099
변경 전 닉네임 : {'뀨우        이잇'}
변경 후 닉네임 : {'13645187del'}
168100
168101
168102
168103
168104
168105
168106
168107
168108
168109
168110
168111
168112
168113
168114
16

168833
168834
168835
168836
168837
168838
168839
168840
168841
168842
168843
168844
168845
168846
168847
168848
168849
168850
168851
168852
168853
168854
168855
168856
168857
168858
168859
168860
168861
168862
168863
168864
168865
168866
168867
168868
168869
168870
168871
168872
168873
168874
168875
168876
168877
168878
168879
168880
168881
168882
168883
168884
168885
168886
168887
168888
168889
168890
168891
168892
168893
168894
168895
168896
168897
168898
168899
168900
168901
168902
168903
168904
168905
168906
168907
168908
168909
168910
168911
168912
오류 발생: 404
오류 발생 위치 : 168912
변경 전 닉네임 : {'멋있는스웨인'}
변경 후 닉네임 : {'너의관심사가뭐니'}
168913
168914
Rate Limit 초과, 90 초 대기 후 다시 시도합니다.
요청 중 오류 발생: name 'headers' is not defined
오류 발생 위치 : 168914
변경 전 닉네임 : {'스노우페더'}
변경 후 닉네임 : {'스노우페더'}
168915
168916
168917
168918
168919
168920
168921
168922
168923
168924
168925
168926
168927
168928
168929
168930
168931
168932
168933
168934
168935
168936
168937
168938
168939
168940
168941
168942
168943
168944
1689

변경 후 닉네임 : {'수도승 훈련'}
169650
169651
169652
169653
169654
169655
169656
169657
169658
169659
169660
169661
169662
169663
169664
169665
169666
169667
169668
169669
169670
Rate Limit 초과, 86 초 대기 후 다시 시도합니다.
요청 중 오류 발생: name 'headers' is not defined
오류 발생 위치 : 169670
변경 전 닉네임 : {'김삐리뽕'}
변경 후 닉네임 : {'김삐리뽕'}
169671
169672
169673
169674
169675
169676
169677
169678
169679
169680
169681
169682
169683
169684
169685
169686
169687
169688
169689
169690
169691
169692
169693
169694
169695
169696
169697
169698
169699
169700
169701
169702
169703
169704
169705
169706
169707
169708
169709
169710
169711
169712
169713
169714
169715
169716
169717
169718
169719
169720
169721
169722
169723
169724
169725
169726
169727
169728
169729
169730
169731
169732
169733
169734
169735
169736
169737
169738
169739
169740
169741
169742
오류 발생: 404
오류 발생 위치 : 169742
변경 전 닉네임 : {'김민드기'}
변경 후 닉네임 : {'102374575del'}
169743
169744
169745
169746
169747
169748
169749
169750
169751
169752
169753
169754
169755
169756
169757
169758
169

변경 후 닉네임 : {'cn0'}
170504
170505
170506
170507
170508
170509
170510
170511
170512
170513
170514
170515
170516
170517
170518
170519
170520
170521
170522
170523
170524
170525
170526
170527
170528
170529
170530
170531
170532
170533
170534
170535
Rate Limit 초과, 86 초 대기 후 다시 시도합니다.
요청 중 오류 발생: name 'headers' is not defined
오류 발생 위치 : 170535
변경 전 닉네임 : {'선배 좋아해요'}
변경 후 닉네임 : {'선배 좋아해요'}
170536
170537
170538
170539
170540
170541
170542
170543
170544
170545
170546
170547
170548
170549
170550
170551
170552
170553
170554
170555
170556
170557
170558
170559
170560
170561
170562
170563
170564
170565
170566
170567
170568
170569
170570
170571
170572
170573
170574
170575
170576
170577
170578
170579
170580
170581
170582
170583
170584
170585
170586
170587
170588
170589
170590
오류 발생: 404
오류 발생 위치 : 170590
변경 전 닉네임 : {'미친남자 김찬호'}
변경 후 닉네임 : {'su cut no yeah'}
170591
170592
170593
170594
170595
170596
170597
170598
170599
170600
170601
170602
170603
170604
170605
170606
170607
170608
170609
오류 발생: 404
오류 발

...

변경 후 닉네임 : {'T1 Fulls'}
207454
207455
207456
오류 발생: 404
오류 발생 위치 : 207456
변경 전 닉네임 : {'고혜린'}
변경 후 닉네임 : {'shadowplay'}
207457
207458
207459
207460
207461
207462
207463
207464
207465
207466
207467
오류 발생: 404
오류 발생 위치 : 207467
변경 전 닉네임 : {'짜 0I 링'}
변경 후 닉네임 : {'짜 E'}
207468
207469
207470
207471
207472
207473
207474
207475
207476
207477
207478
207479
207480
207481
207482
207483
207484
207485
207486
207487
207488
207489
207490
207491
207492
207493
207494
오류 발생: 404
오류 발생 위치 : 207494
변경 전 닉네임 : {'올록볼록브로콜리'}
변경 후 닉네임 : {'무시할정도의강자'}
207495
오류 발생: 404
오류 발생 위치 : 207495
변경 전 닉네임 : {'빛의 메아리'}
변경 후 닉네임 : {'로즈애비'}
207496
207497
207498
오류 발생: 404
오류 발생 위치 : 207498
변경 전 닉네임 : {'절대못이겨15'}
변경 후 닉네임 : {'바론밑으로안감'}
207499
207500
207501
207502
207503
207504
207505
207506
오류 발생: 404
오류 발생 위치 : 207506
변경 전 닉네임 : {'Cloud 12'}
변경 후 닉네임 : {'Amor x'}
207507
Rate Limit 초과, 81 초 대기 후 다시 시도합니다.
요청 중 오류 발생: name 'headers' is not defined
오류 발생 위치 : 207507
변경 전 닉네임 : {'자비를베풀어줄게'}
변경 후 닉네임 : {'자비를베풀어줄게'}
207508
207509

변경 후 닉네임 : {'caijiuduolian1'}
208112
208113
208114
208115
208116
208117
208118
208119
208120
208121
208122
208123
208124
208125
208126
208127
208128
208129
208130
오류 발생: 404
오류 발생 위치 : 208130
변경 전 닉네임 : {'상체와서폿을믿자'}
변경 후 닉네임 : {'내가 죄인인가'}
208131
208132
208133
208134
208135
208136
208137
208138
208139
208140
오류 발생: 404
오류 발생 위치 : 208140
변경 전 닉네임 : {'LuvBreakinApart'}
변경 후 닉네임 : {'BeatBoxDG'}
Rate Limit 초과, 85 초 대기 후 다시 시도합니다.
요청 중 오류 발생: name 'headers' is not defined
닉네임 조회 실패로 summonerId로 조회 실행: BgdrB9KVQxZq9S4K_Fqq41GPw3PgbSCm3GPFinaxwudS1w
208141
208142
208143
208144
208145
208146
208147
208148
오류 발생: 404
오류 발생 위치 : 208148
변경 전 닉네임 : {'그저 혐팬치'}
변경 후 닉네임 : {'Emotional Orange'}
208149
208150
208151
208152
208153
208154
208155
208156
208157
오류 발생: 404
오류 발생 위치 : 208157
변경 전 닉네임 : {'하아 석 나가네'}
변경 후 닉네임 : {'심병준'}
208158
208159
208160
208161
208162
208163
오류 발생: 404
오류 발생 위치 : 208163
변경 전 닉네임 : {'sshn'}
변경 후 닉네임 : {'Guargel'}
208164
오류 발생: 404
오류 발생 위치 : 208164
변경 전 닉네임 : {'겸상 좀 하지마'}
변경 후

208725
208726
208727
208728
오류 발생: 404
오류 발생 위치 : 208728
변경 전 닉네임 : {'NKM53'}
변경 후 닉네임 : {'INE02'}
208729
208730
208731
208732
208733
208734
208735
208736
208737
208738
208739
208740
208741
208742
208743
208744
208745
208746
208747
208748
208749
208750
208751
208752
208753
208754
오류 발생: 404
오류 발생 위치 : 208754
변경 전 닉네임 : {'Theghostisonlyme'}
변경 후 닉네임 : {'Manners maketh m'}
208755
208756
208757
208758
오류 발생: 404
오류 발생 위치 : 208758
변경 전 닉네임 : {'Muryosorasho'}
Rate Limit 초과, 86초 대기 후 재시도
변경 후 닉네임 : {'cagel'}
208759
208760
208761
오류 발생: 404
오류 발생 위치 : 208761
변경 전 닉네임 : {'йot happy'}
변경 후 닉네임 : {'머리가지끈지끈'}
208762
208763
208764
208765
208766
208767
208768
208769
208770
208771
208772
208773
208774
208775
208776
208777
208778
208779
208780
208781
208782
208783
208784
208785
208786
208787
208788
208789
208790
오류 발생: 404
오류 발생 위치 : 208790
변경 전 닉네임 : {'흐끅흐끅'}
변경 후 닉네임 : {'전헌호'}
208791
208792
208793
208794
208795
208796
208797
208798
208799
208800
208801
208802
208803
208804
208805
208806
208807
2088

209467
209468
209469
209470
209471
209472
209473
209474
209475
209476
209477
209478
209479
209480
209481
209482
209483
209484
209485
209486
209487
209488
209489
209490
오류 발생: 404
오류 발생 위치 : 209490
변경 전 닉네임 : {'미드아우솔할래'}
변경 후 닉네임 : {'wefqwqwfqwa'}
209491
209492
209493
209494
209495
209496
209497
209498
209499
209500
209501
209502
오류 발생: 404
오류 발생 위치 : 209502
변경 전 닉네임 : {'qewasdzxc'}
변경 후 닉네임 : {'파오리창파나이트'}
209503
209504
Rate Limit 초과, 89 초 대기 후 다시 시도합니다.
요청 중 오류 발생: name 'headers' is not defined
오류 발생 위치 : 209504
변경 전 닉네임 : {'Karches'}
변경 후 닉네임 : {'Karches'}
209505
209506
209507
209508
209509
209510
209511
209512
209513
209514
209515
209516
209517
209518
209519
209520
209521
209522
209523
209524
209525
209526
209527
209528
209529
209530
209531
209532
209533
209534
209535
209536
209537
209538
209539
209540
209541
209542
209543
209544
209545
209546
209547
209548
209549
209550
209551
209552
209553
209554
209555
209556
209557
209558
209559
209560
209561
오류 발생: 404
오류 발생 위치 : 209561
변경 전 닉네임

변경 후 닉네임 : {'svp king'}
210162
210163
210164
210165
210166
210167
210168
210169
210170
210171
210172
210173
210174
210175
210176
210177
오류 발생: 404
오류 발생 위치 : 210177
변경 전 닉네임 : {'Tae Hyeoй'}
변경 후 닉네임 : {'내게 행복을'}
210178
210179
210180
210181
210182
210183
210184
210185
210186
210187
210188
210189
210190
210191
210192
오류 발생: 404
오류 발생 위치 : 210192
변경 전 닉네임 : {'keep focus'}
변경 후 닉네임 : {'꿈이바로앞에있는'}
210193
210194
210195
210196
210197
210198
210199
210200
210201
210202
210203
210204
210205
210206
210207
210208
210209
210210
210211
210212
210213
210214
210215
210216
210217
210218
오류 발생: 404
오류 발생 위치 : 210218
변경 전 닉네임 : {'셔틀패치'}
변경 후 닉네임 : {'인파이트'}
210219
210220
210221
210222
210223
210224
210225
210226
210227
210228
210229
210230
210231
210232
210233
Rate Limit 초과, 87 초 대기 후 다시 시도합니다.
요청 중 오류 발생: name 'headers' is not defined
오류 발생 위치 : 210233
변경 전 닉네임 : {'당하중학교'}
변경 후 닉네임 : {'당하중학교'}
210234
210235
오류 발생: 404
오류 발생 위치 : 210235
변경 전 닉네임 : {'아무개냥'}
변경 후 닉네임 : {'불빠따 오타니군'}
210236
210237
210238
210

In [24]:
len(datas)

210738

In [70]:
summoner_v4_by_encryptedSummonerId(datas[5]['summonerId'])

{'status': {'message': 'Forbidden', 'status_code': 403}}

In [37]:
summoner_info = get_summoner_info(api_key, '10031917_DEL')

오류 발생: 404


In [68]:
datas[6819]

{'user_name': '이모가핑보라했지',
 'play_num': 39,
 'win_rate': 46.15384615384615,
 'veteran': False,
 'hot_streak': False,
 'freshblood': False,
 'league_points': 26,
 'summonerId': 'EqnWX7KZJMGJwYSf4tRmw4OUVxhIAvdVPNkdYojxlnqcBaI',
 'inactive': False,
 'tier': 'BRONZE'}

In [97]:
summoner_info = get_summoner_info(api_key, datas[17811]['user_name'])

In [29]:
day_list = []

In [41]:
# today_date_in_new_york = get_today_date_in_new_york() # 오늘 날짜(뉴욕 기준)

today_date_in_new_york = convert_milliseconds_to_new_york_time(1694620800000)

for i in range(0, len(datas)):
    
    time_newyork = convert_milliseconds_to_new_york_time(datas[i]['revisionDate'])

    diff_time = today_date_in_new_york - time_newyork    

    days_diff = diff_time.days
    
    day_list.append(days_diff)
    
    if days_diff <= 7:
        datas[i]['leave_game'] = True
        print(days_diff)
    else:
        datas[i]['leave_game'] = False

8
5
13
8
18
17
25
25
18
10
42
9
25
19
9
37
17
31
6
5
48
31
12
7
6
5
34
19
10
13
50
35
8
10
52
5
37
20
31
30
6
43
33
11
7
48
7
34
17
6
5
21
10
26
16
11
10
27
9
11
26
6
17
14
49
24
22
42
23
17
10
24
16
11
5
21
13
45
31
21
19
31
21
19
10
13
11
8
7
16
20
10
13
15
7
30
19
5
17
28
19
6
7
11
16
19
7
9
12
16
6
5
44
34
5
7
41
29
11
6
31
31
25
6
9
24
16
32
11
7
28
19
30
7
25
33
8
7
11
15
11
18
27
14
14
11
14
17
11
39
29
6
40
37
6
45
12
5
12
5
19
17
12
31
13
21
10
13
13
24
10
17
50
6
9
9
31
42
31
25
20
5
15
35
6
7
36
7
8
28
36
10
11
6
17
10
5
15
18
6
6
44
6
36
41
6
6
26
49
24
17
31
13
32
11
28
13
5
10
18
39
18
10
10
34
25
29
6
15
18
17
18
9
12
39
6
26
6
5
12
21
41
12
5
9
6
5
9
6
5
45
29
11
10
10
54
30
10
38
7
30
17
18
12
12
5
10
7
11
8
36
23
7
13
21
32
18
38
5
16
24
32
10
44
13
19
12
38
6
5
10
51
49
10
17
19
38
18
25
37
31
29
6
46
14
33
17
13
13
7
5
33
17
31
47
6
8
12
32
7
18
41
13
12
10
17
37
20
13
31
7
21
14
42
42
5
30
23
28
17
5
6
12
19
19
5
24
44
13
7
9
35
9
54
46
18
10
27
10
9
38
5
23
6
11
3

KeyError: 'revisionDate'

In [36]:
len(datas)

210738

In [47]:
with open('datas_practice_5.pickle', 'wb') as f:
    pickle.dump(datas, f, pickle.HIGHEST_PROTOCOL)

In [35]:
with open('datas_practice_5.pickle', 'rb') as f:
    datas = pickle.load(f)

In [28]:
len(datas)

51250

In [31]:
def get_mastery_info(api_key, region, encrypted_summoner_id):
    # API 엔드포인트 URL 생성
    base_url = f"https://{region}.api.riotgames.com/lol/champion-mastery/v4/champion-masteries/by-summoner/{encrypted_summoner_id}"
    
    # API 요청 헤더 설정
    headers = {
        "X-Riot-Token": api_key,
    }
    
    try:
        # API 요청 보내기
        response = requests.get(base_url, headers=headers)
        
        if response.status_code == 429:
            retry_after = int(response.headers['Retry-After'])
            print(f"Rate Limit 초과, {retry_after}초 대기 후 재시도")
            time.sleep(retry_after)
            response = requests.get(base_url, headers=headers)
        # 응답 확인
        if response.status_code == 200:
            mastery_info = response.json()
            return mastery_info
        else:
            print(f"API 요청 실패. 상태 코드: {response.status_code}")
            return None
    except Exception as e:
        print(f"API 요청 중 오류 발생: {e}")
        return None

## 챔피언 데이터 불러오기

In [27]:
# JSON 파일을 읽어서 데이터를 파이썬 객체로 변환하는 함수
def load_json_file(file_path):
    with open(file_path, "r", encoding='utf-8') as file:
        data = json.load(file)
    return data

# JSON 파일의 경로
json_file_path = "champion_name.json"

# JSON 파일 불러오기
champion_name_data = load_json_file(json_file_path)

In [28]:
champion_data = []

for idx, st in enumerate(champion_name_data['data']):
    champion = {st : champion_name_data['data'][st]['key'], 'type' : champion_name_data['data'][st]['tags']}
    champion_data.append(champion)

In [46]:
for i1 in range(173531, len(datas)):
    
    print(i1)
        
    user_cp_portion_support = 0
    user_cp_portion_fighter = 0
    user_cp_portion_assassin = 0
    user_cp_portion_tank = 0
    user_cp_portion_marksman = 0
    user_cp_portion_mage = 0
    
    ex_1 = get_mastery_info(api_key, 'kr', datas[i1]['summonerId'])
    count = 0
    for k in ex_1:
        if k['championLevel'] >= 6:
            count += 1
        else:
            continue
    datas[i1]['champion_master_num'] = count
    datas[i1]['user_champion_num'] = len(ex_1)
    chest_count = 0
    for i2 in ex_1:
        if i2['chestGranted'] == True:
            chest_count += 1
        for i3 in champion_data:
            if i2['championId'] == int(list(i3.values())[0]):
                for i4 in list(i3.values())[1]:
                    if i4 == 'Assassin':
                        user_cp_portion_assassin += int(i2['championPoints'])
                    elif i4 == 'Mage':
                        user_cp_portion_mage += int(i2['championPoints'])
                    elif i4 == 'Fighter':
                        user_cp_portion_fighter += int(i2['championPoints'])
                    elif i4 == 'Marksman':
                        user_cp_portion_marksman += int(i2['championPoints'])
                    elif i4 == 'Tank':
                        user_cp_portion_tank += int(i2['championPoints'])
                    elif i4 == 'Support':
                        user_cp_portion_support += int(i2['championPoints'])
                        
    sum_portion = user_cp_portion_assassin + user_cp_portion_fighter + user_cp_portion_mage + user_cp_portion_marksman + user_cp_portion_support + user_cp_portion_tank
    
    datas[i1]['user_champion_chest_granted_count'] = chest_count
    datas[i1]['user_cp_portion_fighter'] = round((user_cp_portion_fighter / sum_portion), 2)
    datas[i1]['user_cp_portion_mage'] = round((user_cp_portion_mage / sum_portion), 2)
    datas[i1]['user_cp_portion_marksman'] = round((user_cp_portion_marksman / sum_portion), 2)
    datas[i1]['user_cp_portion_support'] = round((user_cp_portion_support / sum_portion), 2)
    datas[i1]['user_cp_portion_tank'] = round((user_cp_portion_tank / sum_portion), 2)

173531
173532
173533
173534
173535
173536
173537
173538
173539
173540
173541
173542
173543
173544
173545
173546
173547
173548
173549
173550
173551
173552
173553
173554
173555
173556
173557
173558
173559
173560
173561
173562
173563
173564
173565
173566
173567
173568
173569
173570
173571
173572
173573
173574
173575
173576
173577
173578
173579
173580
173581
173582
173583
173584
173585
173586
173587
173588
173589
173590
173591
173592
173593
173594
173595
173596
173597
173598
173599
173600
173601
173602
173603
173604
173605
173606
173607
173608
173609
173610
173611
173612
173613
173614
173615
173616
173617
173618
173619
173620
173621
173622
173623
173624
173625
173626
173627
173628
173629
173630
173631
Rate Limit 초과, 89초 대기 후 재시도
173632
173633
173634
173635
173636
173637
173638
173639
173640
173641
173642
173643
173644
173645
173646
173647
173648
173649
173650
173651
173652
173653
173654
173655
173656
173657
173658
173659
173660
173661
173662
173663
173664
173665
173666
173667
173668
173669

174658
174659
174660
174661
174662
174663
174664
174665
174666
174667
174668
174669
174670
174671
174672
174673
174674
174675
174676
174677
174678
174679
174680
174681
174682
174683
174684
174685
174686
174687
174688
174689
174690
174691
174692
174693
174694
174695
174696
174697
174698
174699
174700
174701
174702
174703
174704
174705
174706
174707
174708
174709
174710
174711
174712
174713
174714
174715
174716
174717
174718
174719
174720
174721
174722
174723
174724
174725
174726
174727
174728
174729
174730
174731
174732
174733
174734
174735
174736
174737
174738
174739
174740
174741
174742
174743
174744
174745
174746
174747
174748
174749
174750
174751
174752
174753
174754
174755
174756
174757
174758
174759
174760
174761
174762
174763
174764
174765
174766
174767
174768
174769
174770
174771
174772
174773
174774
174775
174776
174777
174778
174779
174780
174781
174782
174783
174784
174785
174786
174787
174788
174789
174790
174791
174792
174793
174794
174795
174796
174797
174798
174799
174800

175805
175806
175807
175808
175809
175810
175811
175812
175813
175814
175815
175816
175817
175818
175819
175820
175821
175822
175823
175824
175825
175826
175827
175828
175829
175830
175831
175832
175833
175834
175835
175836
175837
175838
175839
175840
175841
175842
175843
175844
175845
175846
175847
175848
175849
175850
175851
175852
175853
175854
175855
175856
175857
175858
175859
175860
175861
175862
175863
175864
175865
175866
175867
175868
175869
175870
175871
175872
175873
175874
175875
175876
175877
175878
Rate Limit 초과, 81초 대기 후 재시도
175879
175880
175881
175882
175883
175884
175885
175886
175887
175888
175889
175890
175891
175892
175893
175894
175895
175896
175897
175898
175899
175900
175901
175902
175903
175904
175905
175906
175907
175908
175909
175910
175911
175912
175913
175914
175915
175916
175917
175918
175919
175920
175921
175922
175923
175924
175925
175926
175927
175928
175929
175930
175931
175932
175933
175934
175935
175936
175937
175938
175939
175940
175941
175942
175943

176932
176933
176934
176935
176936
176937
176938
176939
176940
176941
176942
176943
176944
176945
176946
176947
176948
176949
176950
176951
176952
176953
176954
176955
176956
176957
176958
176959
176960
176961
176962
176963
176964
176965
176966
176967
176968
176969
176970
176971
176972
176973
176974
176975
176976
176977
176978
Rate Limit 초과, 90초 대기 후 재시도
176979
176980
176981
176982
176983
176984
176985
176986
176987
176988
176989
176990
176991
176992
176993
176994
176995
176996
176997
176998
176999
177000
177001
177002
177003
177004
177005
177006
177007
177008
177009
177010
177011
177012
177013
177014
177015
177016
177017
177018
177019
177020
177021
177022
177023
177024
177025
177026
177027
177028
177029
177030
177031
177032
177033
177034
177035
177036
177037
177038
177039
177040
177041
177042
177043
177044
177045
177046
177047
177048
177049
177050
177051
177052
177053
177054
177055
177056
177057
177058
177059
177060
177061
177062
177063
177064
177065
177066
177067
177068
177069
177070

178059
178060
178061
178062
178063
178064
178065
178066
178067
178068
178069
178070
178071
178072
178073
178074
178075
178076
178077
178078
Rate Limit 초과, 67초 대기 후 재시도
178079
178080
178081
178082
178083
178084
178085
178086
178087
178088
178089
178090
178091
178092
178093
178094
178095
178096
178097
178098
178099
178100
178101
178102
178103
178104
178105
178106
178107
178108
178109
178110
178111
178112
178113
178114
178115
178116
178117
178118
178119
178120
178121
178122
178123
178124
178125
178126
178127
178128
178129
178130
178131
178132
178133
178134
178135
178136
178137
178138
178139
178140
178141
178142
178143
178144
178145
178146
178147
178148
178149
178150
178151
178152
178153
178154
178155
178156
178157
178158
178159
178160
178161
178162
178163
178164
178165
178166
178167
178168
178169
178170
178171
178172
178173
178174
178175
178176
178177
178178
Rate Limit 초과, 74초 대기 후 재시도
178179
178180
178181
178182
178183
178184
178185
178186
178187
178188
178189
178190
178191
178192
178193

...

206206
206207
206208
206209
206210
206211
206212
206213
206214
206215
206216
206217
206218
206219
206220
206221
206222
206223
206224
206225
206226
206227
206228
206229
206230
206231
206232
206233
206234
206235
206236
206237
206238
206239
206240
206241
206242
206243
206244
206245
206246
206247
206248
206249
206250
206251
206252
206253
206254
206255
206256
206257
206258
206259
206260
206261
206262
206263
206264
206265
206266
206267
206268
206269
206270
206271
206272
206273
206274
206275
206276
206277
206278
Rate Limit 초과, 92초 대기 후 재시도
206279
206280
206281
206282
206283
206284
206285
206286
206287
206288
206289
206290
206291
206292
206293
206294
206295
206296
206297
206298
206299
206300
206301
206302
206303
206304
206305
206306
206307
206308
206309
206310
206311
206312
206313
206314
206315
206316
206317
206318
206319
206320
206321
206322
206323
206324
206325
206326
206327
206328
206329
206330
206331
206332
206333
206334
206335
206336
206337
206338
206339
206340
206341
206342
206343
206344

207333
207334
207335
207336
207337
207338
207339
207340
207341
207342
207343
207344
207345
207346
207347
207348
207349
207350
207351
207352
207353
207354
207355
207356
207357
207358
207359
207360
207361
207362
207363
207364
207365
207366
207367
207368
207369
207370
207371
207372
207373
207374
207375
207376
207377
207378
Rate Limit 초과, 93초 대기 후 재시도
207379
207380
207381
207382
207383
207384
207385
207386
207387
207388
207389
207390
207391
207392
207393
207394
207395
207396
207397
207398
207399
207400
207401
207402
207403
207404
207405
207406
207407
207408
207409
207410
207411
207412
207413
207414
207415
207416
207417
207418
207419
207420
207421
207422
207423
207424
207425
207426
207427
207428
207429
207430
207431
207432
207433
207434
207435
207436
207437
207438
207439
207440
207441
207442
207443
207444
207445
207446
207447
207448
207449
207450
207451
207452
207453
207454
207455
207456
207457
207458
207459
207460
207461
207462
207463
207464
207465
207466
207467
207468
207469
207470
207471

208460
208461
208462
208463
208464
208465
208466
208467
208468
208469
208470
208471
208472
208473
208474
208475
208476
208477
208478
Rate Limit 초과, 92초 대기 후 재시도
208479
208480
208481
208482
208483
208484
208485
208486
208487
208488
208489
208490
208491
208492
208493
208494
208495
208496
208497
208498
208499
208500
208501
208502
208503
208504
208505
208506
208507
208508
208509
208510
208511
208512
208513
208514
208515
208516
208517
208518
208519
208520
208521
208522
208523
208524
208525
208526
208527
208528
208529
208530
208531
208532
208533
208534
208535
208536
208537
208538
208539
208540
208541
208542
208543
208544
208545
208546
208547
208548
208549
208550
208551
208552
208553
208554
208555
208556
208557
208558
208559
208560
208561
208562
208563
208564
208565
208566
208567
208568
208569
208570
208571
208572
208573
208574
208575
208576
208577
208578
Rate Limit 초과, 92초 대기 후 재시도
208579
208580
208581
208582
208583
208584
208585
208586
208587
208588
208589
208590
208591
208592
208593
208594

209583
209584
209585
209586
209587
209588
209589
209590
209591
209592
209593
209594
209595
209596
209597
209598
209599
209600
209601
209602
209603
209604
209605
209606
209607
209608
209609
209610
209611
209612
209613
209614
209615
209616
209617
209618
209619
209620
209621
209622
209623
209624
209625
209626
209627
209628
209629
209630
209631
209632
209633
209634
209635
209636
209637
209638
209639
209640
209641
209642
209643
209644
209645
209646
209647
209648
209649
209650
209651
209652
209653
209654
209655
209656
209657
209658
209659
209660
209661
209662
209663
209664
209665
209666
209667
209668
209669
209670
209671
209672
209673
209674
209675
209676
209677
209678
Rate Limit 초과, 93초 대기 후 재시도
209679
209680
209681
209682
209683
209684
209685
209686
209687
209688
209689
209690
209691
209692
209693
209694
209695
209696
209697
209698
209699
209700
209701
209702
209703
209704
209705
209706
209707
209708
209709
209710
209711
209712
209713
209714
209715
209716
209717
209718
209719
209720
209721

210710
210711
210712
210713
210714
210715
210716
210717
210718
210719
210720
210721
210722
210723
210724
210725
210726
210727
210728
210729
210730
210731
210732
210733
210734
210735
210736
210737


In [48]:
datas[210737]

{'user_name': '의문의 버스승객',
 'play_num': 39,
 'win_rate': 53.84615384615385,
 'veteran': False,
 'hot_streak': False,
 'freshblood': False,
 'league_points': 0,
 'summonerId': 'wkNXKehwJ6hAesdCCYnUNsHlb_s9OHM76ninH27nJZy26g',
 'inactive': False,
 'tier': 'MASTER',
 'revisionDate': 1697457412000,
 'summonerLevel': 388,
 'puuid': 'z5AGALpfGanmdjWTJqDWJvI9sOPWo7bSYddbD_H8pKssvsk6OUcQQ8oceY3-MOTW-c-XWW6Kpiahag',
 'champion_master_num': 31,
 'user_champion_num': 130,
 'user_champion_chest_granted_count': 14,
 'user_cp_portion_fighter': 0.28,
 'user_cp_portion_mage': 0.21,
 'user_cp_portion_marksman': 0.06,
 'user_cp_portion_support': 0.08,
 'user_cp_portion_tank': 0.23}

In [52]:
with open('datas_practice_5.pickle', 'wb') as f:
    pickle.dump(datas, f, pickle.HIGHEST_PROTOCOL)

In [24]:
datas[51249]

{'user_name': 'Monaka',
 'play_num': 557,
 'win_rate': 50.80789946140036,
 'veteran': True,
 'hot_streak': False,
 'freshblood': False,
 'league_points': 60,
 'summonerId': 'ocAgauVauVyrGycpfHxHpQE22rKjbZoMWNChzqVw81xWDEU',
 'inactive': False,
 'tier': 'MASTER',
 'revisionDate': 1695292942000,
 'summonerLevel': 336,
 'puuid': 'u5x2ImtL7fp7t02Grf2glzaLSzlux2725m9v9u6yGkUb77hGt0XrP9lKJJ14un0j0AELDePi7YjdQA'}

## 종속변수 계산

In [114]:
day_list = []

In [150]:
# today_date_in_new_york = get_today_date_in_new_york() # 오늘 날짜(뉴욕 기준)

today_date_in_new_york = convert_milliseconds_to_new_york_time(1694620800000)

for i in range(0, len(datas)):
    
    time_newyork = convert_milliseconds_to_new_york_time(datas[i]['revisionDate'])

    diff_time = today_date_in_new_york - time_newyork    

    days_diff = diff_time.days
    
    day_list.append(days_diff)
    
    if days_diff <= 7:
        datas[i]['leave_game'] = 0
    elif days_diff > 7: 
        datas[i]['leave_game'] = 1

In [50]:
with open('datas_practice_5.pickle', 'rb') as f:
    datas = pickle.load(f)

In [51]:
for i in range(0, len(datas)):
    if datas[i]['tier'] == 'IRON':
        datas[i]['tier_points'] = 1
    elif datas[i]['tier'] == 'BRONZE':
        datas[i]['tier_points'] = 2
    elif datas[i]['tier'] == 'SILVER':
        datas[i]['tier_points'] = 3
    elif datas[i]['tier'] == 'GOLD':
        datas[i]['tier_points'] = 4
    elif datas[i]['tier'] == 'PLATINUM':
        datas[i]['tier_points'] = 5
    elif datas[i]['tier'] == 'EMERALD':
        datas[i]['tier_points'] = 6
    elif datas[i]['tier'] == 'DIAMOND':
        datas[i]['tier_points'] = 7
    elif datas[i]['tier'] == 'MASTER':
        datas[i]['tier_points'] = 8
    elif datas[i]['tier'] == 'GRANDMASTER':
        datas[i]['tier_points'] = 9
    elif datas[i]['tier'] == 'CHALLENGER':
        datas[i]['tier_points'] = 10

In [31]:
datas[25624]

{'user_name': '나도 여친 내놔',
 'play_num': 72,
 'win_rate': 58.333333333333336,
 'veteran': False,
 'hot_streak': False,
 'freshblood': False,
 'league_points': 34,
 'summonerId': '4vrhyCoEYvnZx_aegPhgGiCd6gciBpVCYXsWr9d66ancDdGq',
 'inactive': False,
 'tier': 'GOLD',
 'revisionDate': 1694888656000,
 'summonerLevel': 156,
 'puuid': '1TaMMLRJHjdG0DRoMxGSsXMrrYOsPlvQqNMX-dVLpL_EJII8a1UFDockfIbFnUuay_wuBQIiDQVopA',
 'champion_master_num': 7,
 'user_champion_num': 72,
 'user_champion_chest_granted_count': 9,
 'user_cp_portion_fighter': 0.06,
 'user_cp_portion_mage': 0.12,
 'user_cp_portion_marksman': 0.59,
 'user_cp_portion_support': 0.12,
 'user_cp_portion_tank': 0.03,
 'champion_roll_num': 3,
 'tier_points': 4}

In [63]:
df_x_train

NameError: name 'df_x_train' is not defined